In [21]:
import os
import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from tqdm import tqdm

# === PATH SETUP ===
VILLAGE_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_districts"
PHC_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_Facilities"
OUTPUT_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === DISTRICTS: First 10 alphabetically ===
districts = ['anugul', 'balangir', 'baleshwar', 'bargarh', 'baudh', 
             'bhadrak', 'cuttack', 'deogarh', 'dhenkanal', 'gajapati']

# === OSRM routing function ===
def osrm_route(src, dst):
    url = f"http://localhost:5000/route/v1/driving/{src.x},{src.y};{dst.x},{dst.y}?overview=false"
    try:
        r = requests.get(url)
        data = r.json()
        if r.status_code == 200 and 'routes' in data:
            duration = data['routes'][0]['duration']  # in seconds
            distance = data['routes'][0]['distance']  # in meters
            return duration, distance
    except:
        return None, None
    return None, None

# === PROCESS EACH DISTRICT ===
for district in districts:
    print(f"\n🚀 Processing: {district.upper()}")

    # === Load village and PHC files ===
    v_path = os.path.join(VILLAGE_DIR, f"{district}_villages_snapped.geojson")
    f_path = os.path.join(PHC_DIR, f"{district}_phc.geojson")

    if not os.path.exists(v_path) or not os.path.exists(f_path):
        print(f"❌ Missing files for {district}, skipping...")
        continue

    villages = gpd.read_file(v_path).to_crs(epsg=4326)
    phcs = gpd.read_file(f_path).to_crs(epsg=4326)

    # === Prepare outputs ===
    output_rows = []

    for i, v_row in tqdm(villages.iterrows(), total=len(villages), desc=f"⏱ Routing villages in {district}"):
        v_geom = v_row.geometry
        v_id = v_row.get('village_id', i)

        # Find top 3 nearest PHCs by straight-line distance
        phcs['euclid_dist'] = phcs.geometry.distance(v_geom)
        nearest = phcs.sort_values('euclid_dist').head(3)

        best_time = float('inf')
        best_record = None

        for _, f_row in nearest.iterrows():
            f_geom = f_row.geometry
            duration, distance = osrm_route(v_geom, f_geom)
            if duration is not None:
                if duration < best_time:
                    best_time = duration
                    best_record = {
                        'village_id': v_id,
                        'facility_id': f_row.get('facilityname', ''),
                        'facility_type': 'phc',
                        'duration_sec': duration,
                        'distance_m': distance,
                        'facility_lat': f_geom.y,
                        'facility_lon': f_geom.x
                    }

        if best_record:
            output_rows.append(best_record)

    # === Save output ===
    df_out = pd.DataFrame(output_rows)
    out_path = os.path.join(OUTPUT_DIR, f"{district}_village_to_phc.csv")
    df_out.to_csv(out_path, index=False)
    print(f"✅ Saved: {out_path} [{len(df_out)} records]")


🚀 Processing: ANUGUL


⏱ Routing villages in anugul:   0%|          | 0/1992 [00:00<?, ?it/s]C:\Users\utkar\AppData\Local\Temp\ipykernel_78684\1892907156.py:55: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  phcs['euclid_dist'] = phcs.geometry.distance(v_geom)
⏱ Routing villages in anugul:   0%|          | 1/1992 [00:00<12:09,  2.73it/s]C:\Users\utkar\AppData\Local\Temp\ipykernel_78684\1892907156.py:55: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  phcs['euclid_dist'] = phcs.geometry.distance(v_geom)
⏱ Routing villages in anugul: 100%|██████████| 1992/1992 [01:39<00:00, 20.06it/s]


✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\anugul_village_to_phc.csv [1992 records]

🚀 Processing: BALANGIR


⏱ Routing villages in balangir:   0%|          | 0/1874 [00:00<?, ?it/s]C:\Users\utkar\AppData\Local\Temp\ipykernel_78684\1892907156.py:55: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  phcs['euclid_dist'] = phcs.geometry.distance(v_geom)
C:\Users\utkar\AppData\Local\Temp\ipykernel_78684\1892907156.py:55: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  phcs['euclid_dist'] = phcs.geometry.distance(v_geom)
⏱ Routing villages in balangir: 100%|██████████| 1874/1874 [01:32<00:00, 20.19it/s]


✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\balangir_village_to_phc.csv [1874 records]

🚀 Processing: BALESHWAR


...

✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\deogarh_village_to_phc.csv [906 records]

🚀 Processing: DHENKANAL


⏱ Routing villages in dhenkanal:   0%|          | 0/1248 [00:00<?, ?it/s]C:\Users\utkar\AppData\Local\Temp\ipykernel_78684\1892907156.py:55: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  phcs['euclid_dist'] = phcs.geometry.distance(v_geom)
C:\Users\utkar\AppData\Local\Temp\ipykernel_78684\1892907156.py:55: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  phcs['euclid_dist'] = phcs.geometry.distance(v_geom)
⏱ Routing villages in dhenkanal: 100%|██████████| 1248/1248 [01:00<00:00, 20.65it/s]


✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\dhenkanal_village_to_phc.csv [1248 records]

🚀 Processing: GAJAPATI


⏱ Routing villages in gajapati:   0%|          | 0/1646 [00:00<?, ?it/s]C:\Users\utkar\AppData\Local\Temp\ipykernel_78684\1892907156.py:55: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  phcs['euclid_dist'] = phcs.geometry.distance(v_geom)
C:\Users\utkar\AppData\Local\Temp\ipykernel_78684\1892907156.py:55: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  phcs['euclid_dist'] = phcs.geometry.distance(v_geom)
⏱ Routing villages in gajapati: 100%|██████████| 1646/1646 [01:20<00:00, 20.51it/s]

✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\gajapati_village_to_phc.csv [1646 records]


In [22]:
import os
import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from tqdm import tqdm

# === PATH SETUP ===
VILLAGE_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_districts"
PHC_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_Facilities"
OUTPUT_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === DISTRICTS: Next 10 + Baudh ===
districts = [
    'ganjam', 'jagatsinghapur', 'jajapur', 'jharsuguda', 'kalahandi',
    'kandhamal', 'kendrapara', 'keonjhar', 'khordha', 'koraput',
    'baudh'  # Re-added baudh
]

# === OSRM routing function ===
def osrm_route(src, dst):
    url = f"http://localhost:5000/route/v1/driving/{src.x},{src.y};{dst.x},{dst.y}?overview=false"
    try:
        r = requests.get(url)
        data = r.json()
        if r.status_code == 200 and 'routes' in data:
            duration = data['routes'][0]['duration']  # seconds
            distance = data['routes'][0]['distance']  # meters
            return duration, distance
    except:
        return None, None
    return None, None

# === PROCESS EACH DISTRICT ===
for district in districts:
    print(f"\n🚀 Processing: {district.upper()}")

    # Load village and PHC files
    v_path = os.path.join(VILLAGE_DIR, f"{district}_villages_snapped.geojson")
    f_path = os.path.join(PHC_DIR, f"{district}_phc.geojson")

    if not os.path.exists(v_path) or not os.path.exists(f_path):
        print(f"❌ Missing files for {district}, skipping...")
        continue

    villages = gpd.read_file(v_path).to_crs(epsg=4326)
    phcs = gpd.read_file(f_path).to_crs(epsg=4326)

    # Output holder
    output_rows = []

    for i, v_row in tqdm(villages.iterrows(), total=len(villages), desc=f"⏱ Routing villages in {district}"):
        v_geom = v_row.geometry
        v_id = v_row.get('village_id', i)

        # Projected distance calc (to avoid CRS warning)
        phcs_proj = phcs.to_crs(epsg=32644)
        v_geom_proj = gpd.GeoSeries([v_geom], crs=4326).to_crs(epsg=32644).iloc[0]
        phcs_proj['euclid_dist'] = phcs_proj.geometry.distance(v_geom_proj)

        # Get top 3 by Euclidean, original order
        nearest = phcs.loc[phcs_proj.sort_values('euclid_dist').index[:3]]

        best_time = float('inf')
        best_record = None

        for _, f_row in nearest.iterrows():
            f_geom = f_row.geometry
            duration, distance = osrm_route(v_geom, f_geom)
            if duration is not None and duration < best_time:
                best_time = duration
                best_record = {
                    'village_id': v_id,
                    'facility_id': f_row.get('facilityname', ''),
                    'facility_type': 'phc',
                    'duration_sec': duration,
                    'distance_m': distance,
                    'facility_lat': f_geom.y,
                    'facility_lon': f_geom.x
                }

        if best_record:
            output_rows.append(best_record)

    # Save output
    df_out = pd.DataFrame(output_rows)
    out_path = os.path.join(OUTPUT_DIR, f"{district}_village_to_phc.csv")
    df_out.to_csv(out_path, index=False)
    print(f"✅ Saved: {out_path} [{len(df_out)} records]")


🚀 Processing: GANJAM


⏱ Routing villages in ganjam: 100%|██████████| 3328/3328 [02:06<00:00, 26.22it/s]


✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\ganjam_village_to_phc.csv [3328 records]

🚀 Processing: JAGATSINGHAPUR
❌ Missing files for jagatsinghapur, skipping...

🚀 Processing: JAJAPUR


⏱ Routing villages in jajapur: 100%|██████████| 1796/1796 [01:10<00:00, 25.53it/s]


✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\jajapur_village_to_phc.csv [1796 records]

🚀 Processing: JHARSUGUDA


...

✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\keonjhar_village_to_phc.csv [2181 records]

🚀 Processing: KHORDHA


⏱ Routing villages in khordha: 100%|██████████| 1547/1547 [00:57<00:00, 26.83it/s]


✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\khordha_village_to_phc.csv [1547 records]

🚀 Processing: KORAPUT


⏱ Routing villages in koraput: 100%|██████████| 2227/2227 [01:22<00:00, 26.88it/s]

✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\koraput_village_to_phc.csv [2227 records]

🚀 Processing: BAUDH
❌ Missing files for baudh, skipping...


In [23]:
import os
import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from tqdm import tqdm

# === PATH SETUP ===
VILLAGE_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_districts"
PHC_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_Facilities"
OUTPUT_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === FINAL DISTRICT BATCH ===
districts = [
    'malkangiri', 'mayurbhanj', 'nabarangpur', 'nayagarh', 'nuapada',
    'puri', 'rayagada', 'sambalpur', 'sonapur', 'sundargarh'
]

# === OSRM routing function ===
def osrm_route(src, dst):
    url = f"http://localhost:5000/route/v1/driving/{src.x},{src.y};{dst.x},{dst.y}?overview=false"
    try:
        r = requests.get(url)
        data = r.json()
        if r.status_code == 200 and 'routes' in data:
            duration = data['routes'][0]['duration']
            distance = data['routes'][0]['distance']
            return duration, distance
    except:
        return None, None
    return None, None

# === PROCESS EACH DISTRICT ===
for district in districts:
    print(f"\n🚀 Processing: {district.upper()}")

    v_path = os.path.join(VILLAGE_DIR, f"{district}_villages_snapped.geojson")
    f_path = os.path.join(PHC_DIR, f"{district}_phc.geojson")

    if not os.path.exists(v_path) or not os.path.exists(f_path):
        print(f"❌ Missing files for {district}, skipping...")
        continue

    villages = gpd.read_file(v_path).to_crs(epsg=4326)
    phcs = gpd.read_file(f_path).to_crs(epsg=4326)

    output_rows = []

    for i, v_row in tqdm(villages.iterrows(), total=len(villages), desc=f"⏱ Routing villages in {district}"):
        v_geom = v_row.geometry
        v_id = v_row.get('village_id', i)

        # Reproject for accurate Euclidean distance
        phcs_proj = phcs.to_crs(epsg=32644)
        v_geom_proj = gpd.GeoSeries([v_geom], crs=4326).to_crs(epsg=32644).iloc[0]
        phcs_proj['euclid_dist'] = phcs_proj.geometry.distance(v_geom_proj)

        nearest = phcs.loc[phcs_proj.sort_values('euclid_dist').index[:3]]

        best_time = float('inf')
        best_record = None

        for _, f_row in nearest.iterrows():
            f_geom = f_row.geometry
            duration, distance = osrm_route(v_geom, f_geom)
            if duration is not None and duration < best_time:
                best_time = duration
                best_record = {
                    'village_id': v_id,
                    'facility_id': f_row.get('facilityname', ''),
                    'facility_type': 'phc',
                    'duration_sec': duration,
                    'distance_m': distance,
                    'facility_lat': f_geom.y,
                    'facility_lon': f_geom.x
                }

        if best_record:
            output_rows.append(best_record)

    df_out = pd.DataFrame(output_rows)
    out_path = os.path.join(OUTPUT_DIR, f"{district}_village_to_phc.csv")
    df_out.to_csv(out_path, index=False)
    print(f"✅ Saved: {out_path} [{len(df_out)} records]")


🚀 Processing: MALKANGIRI


⏱ Routing villages in malkangiri: 100%|██████████| 1142/1142 [00:42<00:00, 26.67it/s]


✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\malkangiri_village_to_phc.csv [1142 records]

🚀 Processing: MAYURBHANJ


⏱ Routing villages in mayurbhanj: 100%|██████████| 3975/3975 [02:23<00:00, 27.63it/s]


✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\mayurbhanj_village_to_phc.csv [3975 records]

🚀 Processing: NABARANGPUR
❌ Missing files for nabarangpur, skipping...

🚀 Processing: NAYAGARH


...

✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\rayagada_village_to_phc.csv [2824 records]

🚀 Processing: SAMBALPUR


⏱ Routing villages in sambalpur: 100%|██████████| 1414/1414 [00:56<00:00, 25.04it/s]


✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\sambalpur_village_to_phc.csv [1414 records]

🚀 Processing: SONAPUR
❌ Missing files for sonapur, skipping...

🚀 Processing: SUNDARGARH


⏱ Routing villages in sundargarh: 100%|██████████| 1931/1931 [01:20<00:00, 23.88it/s]


✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\sundargarh_village_to_phc.csv [1931 records]


In [24]:
import os
import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from tqdm import tqdm

# === PATHS ===
VILLAGE_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_districts"
PHC_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_Facilities"
OUTPUT_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === CORRECTED DISTRICT KEYS ===
districts = ['boudh', 'kendujhar', 'jagatsinghapur', 'nabarangpur', 'sonepur']

# === Manual overrides for mismatched PHC filenames ===
facility_override = {
    'boudh': 'baudh_phc.geojson',
    'jagatsinghapur': 'jagatsinghpur_phc.geojson',
    'nabarangpur': 'nabarangapur_phc.geojson'
}

# === OSRM ROUTING FUNCTION ===
def osrm_route(src, dst):
    url = f"http://localhost:5000/route/v1/driving/{src.x},{src.y};{dst.x},{dst.y}?overview=false"
    try:
        r = requests.get(url)
        data = r.json()
        if r.status_code == 200 and 'routes' in data:
            duration = data['routes'][0]['duration']
            distance = data['routes'][0]['distance']
            return duration, distance
    except:
        return None, None
    return None, None

# === RUN ROUTING FOR EACH DISTRICT ===
for district in districts:
    print(f"\n🚀 Processing: {district.upper()}")

    v_path = os.path.join(VILLAGE_DIR, f"{district}_villages_snapped.geojson")
    f_name = facility_override.get(district, f"{district}_phc.geojson")
    f_path = os.path.join(PHC_DIR, f_name)

    if not os.path.exists(v_path) or not os.path.exists(f_path):
        print(f"❌ Missing files for {district}, skipping...")
        continue

    villages = gpd.read_file(v_path).to_crs(epsg=4326)
    phcs = gpd.read_file(f_path).to_crs(epsg=4326)

    output_rows = []

    for i, v_row in tqdm(villages.iterrows(), total=len(villages), desc=f"⏱ Routing villages in {district}"):
        v_geom = v_row.geometry
        v_id = v_row.get('village_id', i)

        # Project geometries for Euclidean filtering
        phcs_proj = phcs.to_crs(epsg=32644)
        v_geom_proj = gpd.GeoSeries([v_geom], crs=4326).to_crs(epsg=32644).iloc[0]
        phcs_proj['euclid_dist'] = phcs_proj.geometry.distance(v_geom_proj)

        # Select top 3 by straight-line distance
        nearest = phcs.loc[phcs_proj.sort_values('euclid_dist').index[:3]]

        best_time = float('inf')
        best_record = None

        for _, f_row in nearest.iterrows():
            f_geom = f_row.geometry
            duration, distance = osrm_route(v_geom, f_geom)
            if duration is not None and duration < best_time:
                best_time = duration
                best_record = {
                    'village_id': v_id,
                    'facility_id': f_row.get('facilityname', ''),
                    'facility_type': 'phc',
                    'duration_sec': duration,
                    'distance_m': distance,
                    'facility_lat': f_geom.y,
                    'facility_lon': f_geom.x
                }

        if best_record:
            output_rows.append(best_record)

    df_out = pd.DataFrame(output_rows)
    out_path = os.path.join(OUTPUT_DIR, f"{district}_village_to_phc.csv")
    df_out.to_csv(out_path, index=False)
    print(f"✅ Saved: {out_path} [{len(df_out)} records]")


🚀 Processing: BOUDH
❌ Missing files for boudh, skipping...

🚀 Processing: KENDUJHAR
❌ Missing files for kendujhar, skipping...

🚀 Processing: JAGATSINGHAPUR
❌ Missing files for jagatsinghapur, skipping...

🚀 Processing: NABARANGPUR
❌ Missing files for nabarangpur, skipping...

🚀 Processing: SONEPUR


⏱ Routing villages in sonepur: 100%|██████████| 1000/1000 [00:38<00:00, 26.07it/s]

✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\sonepur_village_to_phc.csv [1000 records]


In [25]:
import os
import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from tqdm import tqdm

# === PATHS ===
VILLAGE_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_districts"
PHC_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_Facilities"
OUTPUT_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === FINAL 2 DISTRICTS ===
districts = ['keonjhar', 'sonepur']

# === OSRM ROUTING FUNCTION ===
def osrm_route(src, dst):
    url = f"http://localhost:5000/route/v1/driving/{src.x},{src.y};{dst.x},{dst.y}?overview=false"
    try:
        r = requests.get(url)
        data = r.json()
        if r.status_code == 200 and 'routes' in data:
            duration = data['routes'][0]['duration']
            distance = data['routes'][0]['distance']
            return duration, distance
    except:
        return None, None
    return None, None

# === ROUTING FOR EACH DISTRICT ===
for district in districts:
    print(f"\n🚀 Processing: {district.upper()}")

    v_path = os.path.join(VILLAGE_DIR, f"{district}_villages_snapped.geojson")
    f_path = os.path.join(PHC_DIR, f"{district}_phc.geojson")

    if not os.path.exists(v_path) or not os.path.exists(f_path):
        print(f"❌ Missing files for {district}, skipping...")
        continue

    villages = gpd.read_file(v_path).to_crs(epsg=4326)
    phcs = gpd.read_file(f_path).to_crs(epsg=4326)

    output_rows = []

    for i, v_row in tqdm(villages.iterrows(), total=len(villages), desc=f"⏱ Routing villages in {district}"):
        v_geom = v_row.geometry
        v_id = v_row.get('village_id', i)

        # Project geometries for Euclidean filtering
        phcs_proj = phcs.to_crs(epsg=32644)
        v_geom_proj = gpd.GeoSeries([v_geom], crs=4326).to_crs(epsg=32644).iloc[0]
        phcs_proj['euclid_dist'] = phcs_proj.geometry.distance(v_geom_proj)

        # Top 3 by Euclidean distance
        nearest = phcs.loc[phcs_proj.sort_values('euclid_dist').index[:3]]

        best_time = float('inf')
        best_record = None

        for _, f_row in nearest.iterrows():
            f_geom = f_row.geometry
            duration, distance = osrm_route(v_geom, f_geom)
            if duration is not None and duration < best_time:
                best_time = duration
                best_record = {
                    'village_id': v_id,
                    'facility_id': f_row.get('facilityname', ''),
                    'facility_type': 'phc',
                    'duration_sec': duration,
                    'distance_m': distance,
                    'facility_lat': f_geom.y,
                    'facility_lon': f_geom.x
                }

        if best_record:
            output_rows.append(best_record)

    df_out = pd.DataFrame(output_rows)
    out_path = os.path.join(OUTPUT_DIR, f"{district}_village_to_phc.csv")
    df_out.to_csv(out_path, index=False)
    print(f"✅ Saved: {out_path} [{len(df_out)} records]")


🚀 Processing: KEONJHAR


⏱ Routing villages in keonjhar: 100%|██████████| 2181/2181 [01:19<00:00, 27.56it/s]


✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\keonjhar_village_to_phc.csv [2181 records]

🚀 Processing: SONEPUR


⏱ Routing villages in sonepur: 100%|██████████| 1000/1000 [00:35<00:00, 28.17it/s]

✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\sonepur_village_to_phc.csv [1000 records]


In [26]:
import os
import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from tqdm import tqdm

# === PATHS ===
VILLAGE_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_districts"
FACILITY_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_Facilities"
OUTPUT_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === ALL 30 DISTRICTS ===
districts = [
    'anugul', 'balangir', 'baleshwar', 'bargarh', 'boudh',
    'bhadrak', 'cuttack', 'deogarh', 'dhenkanal', 'gajapati',
    'ganjam', 'jagatsinghapur', 'jajapur', 'jharsuguda', 'kalahandi',
    'kandhamal', 'kendrapara', 'keonjhar', 'khordha', 'koraput',
    'malkangiri', 'mayurbhanj', 'nabarangpur', 'nayagarh', 'nuapada',
    'puri', 'rayagada', 'sambalpur', 'sonepur', 'sundargarh'
]

# === OSRM ROUTING FUNCTION ===
def osrm_route(src, dst):
    url = f"http://localhost:5000/route/v1/driving/{src.x},{src.y};{dst.x},{dst.y}?overview=false"
    try:
        r = requests.get(url)
        data = r.json()
        if r.status_code == 200 and 'routes' in data:
            duration = data['routes'][0]['duration']
            distance = data['routes'][0]['distance']
            return duration, distance
    except:
        return None, None
    return None, None

# === ROUTE ALL DISTRICTS ===
for district in districts:
    print(f"\n🚀 Processing: {district.upper()}")

    v_path = os.path.join(VILLAGE_DIR, f"{district}_villages_snapped.geojson")
    f_path = os.path.join(FACILITY_DIR, f"{district}_sub_cen.geojson")

    if not os.path.exists(v_path) or not os.path.exists(f_path):
        print(f"❌ Missing files for {district}, skipping...")
        continue

    villages = gpd.read_file(v_path).to_crs(epsg=4326)
    sub_cens = gpd.read_file(f_path).to_crs(epsg=4326)

    output_rows = []

    for i, v_row in tqdm(villages.iterrows(), total=len(villages), desc=f"⏱ Routing villages in {district}"):
        v_geom = v_row.geometry
        v_id = v_row.get('village_id', i)

        # Euclidean distance filter (projected)
        sub_cens_proj = sub_cens.to_crs(epsg=32644)
        v_geom_proj = gpd.GeoSeries([v_geom], crs=4326).to_crs(epsg=32644).iloc[0]
        sub_cens_proj['euclid_dist'] = sub_cens_proj.geometry.distance(v_geom_proj)

        nearest = sub_cens.loc[sub_cens_proj.sort_values('euclid_dist').index[:3]]

        best_time = float('inf')
        best_record = None

        for _, f_row in nearest.iterrows():
            f_geom = f_row.geometry
            duration, distance = osrm_route(v_geom, f_geom)
            if duration is not None and duration < best_time:
                best_time = duration
                best_record = {
                    'village_id': v_id,
                    'facility_id': f_row.get('facilityname', ''),
                    'facility_type': 'sub_cen',
                    'duration_sec': duration,
                    'distance_m': distance,
                    'facility_lat': f_geom.y,
                    'facility_lon': f_geom.x
                }

        if best_record:
            output_rows.append(best_record)

    df_out = pd.DataFrame(output_rows)
    out_path = os.path.join(OUTPUT_DIR, f"{district}_village_to_sub_cen.csv")
    df_out.to_csv(out_path, index=False)
    print(f"✅ Saved: {out_path} [{len(df_out)} records]")


🚀 Processing: ANUGUL


⏱ Routing villages in anugul: 100%|██████████| 1992/1992 [01:10<00:00, 28.37it/s]


✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\anugul_village_to_sub_cen.csv [1992 records]

🚀 Processing: BALANGIR


⏱ Routing villages in balangir: 100%|██████████| 1874/1874 [01:15<00:00, 24.78it/s]


✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\balangir_village_to_sub_cen.csv [1874 records]

🚀 Processing: BALESHWAR


...

✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\sambalpur_village_to_sub_cen.csv [1414 records]

🚀 Processing: SONEPUR


⏱ Routing villages in sonepur: 100%|██████████| 1000/1000 [00:40<00:00, 24.84it/s]


✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\sonepur_village_to_sub_cen.csv [1000 records]

🚀 Processing: SUNDARGARH


⏱ Routing villages in sundargarh: 100%|██████████| 1931/1931 [01:13<00:00, 26.43it/s]

✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\sundargarh_village_to_sub_cen.csv [1931 records]


In [27]:
import os
import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from tqdm import tqdm

# === PATHS ===
VILLAGE_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_districts"
FACILITY_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_Facilities"
OUTPUT_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === DISTRICTS ===
districts = ['nabarangpur', 'boudh', 'jagatsinghapur']

# === Manual file overrides ===
facility_files = {
    'nabarangpur': 'nabarangapur_sub_cen.geojson',
    'boudh': 'boudh_sub_cen.geojson',
    'jagatsinghapur': 'jagatsinghpur_sub_cen.geojson',
}

# === OSRM routing function ===
def osrm_route(src, dst):
    url = f"http://localhost:5000/route/v1/driving/{src.x},{src.y};{dst.x},{dst.y}?overview=false"
    try:
        r = requests.get(url)
        data = r.json()
        if r.status_code == 200 and 'routes' in data:
            return data['routes'][0]['duration'], data['routes'][0]['distance']
    except:
        return None, None
    return None, None

# === Process each district ===
for district in districts:
    print(f"\n🚀 Processing: {district.upper()}")

    v_path = os.path.join(VILLAGE_DIR, f"{district}_villages_snapped.geojson")
    f_path = os.path.join(FACILITY_DIR, facility_files[district])

    if not os.path.exists(v_path) or not os.path.exists(f_path):
        print(f"❌ Missing files for {district}, skipping...")
        continue

    villages = gpd.read_file(v_path).to_crs(epsg=4326)
    sub_cens = gpd.read_file(f_path).to_crs(epsg=4326)

    output_rows = []

    for i, v_row in tqdm(villages.iterrows(), total=len(villages), desc=f"⏱ Routing villages in {district}"):
        v_geom = v_row.geometry
        v_id = v_row.get('village_id', i)

        # Project to metric CRS for Euclidean filtering
        sub_proj = sub_cens.to_crs(epsg=32644)
        v_proj = gpd.GeoSeries([v_geom], crs=4326).to_crs(epsg=32644).iloc[0]
        sub_proj['euclid_dist'] = sub_proj.geometry.distance(v_proj)

        nearest = sub_cens.loc[sub_proj.sort_values('euclid_dist').index[:3]]

        best_time = float('inf')
        best_record = None

        for _, f_row in nearest.iterrows():
            f_geom = f_row.geometry
            duration, distance = osrm_route(v_geom, f_geom)
            if duration is not None and duration < best_time:
                best_time = duration
                best_record = {
                    'village_id': v_id,
                    'facility_id': f_row.get('facilityname', ''),
                    'facility_type': 'sub_cen',
                    'duration_sec': duration,
                    'distance_m': distance,
                    'facility_lat': f_geom.y,
                    'facility_lon': f_geom.x
                }

        if best_record:
            output_rows.append(best_record)

    df_out = pd.DataFrame(output_rows)
    out_path = os.path.join(OUTPUT_DIR, f"{district}_village_to_sub_cen.csv")
    df_out.to_csv(out_path, index=False)
    print(f"✅ Saved: {out_path} [{len(df_out)} records]")


🚀 Processing: NABARANGPUR
❌ Missing files for nabarangpur, skipping...

🚀 Processing: BOUDH


⏱ Routing villages in boudh: 100%|██████████| 1233/1233 [00:45<00:00, 27.06it/s]

✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\boudh_village_to_sub_cen.csv [1233 records]

🚀 Processing: JAGATSINGHAPUR
❌ Missing files for jagatsinghapur, skipping...


In [28]:
import os
import requests
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

# === PATHS ===
VILLAGE_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_districts"
FACILITY_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_Facilities"
OUTPUT_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts"
os.makedirs(OUTPUT_DIR, exist_ok=True)

districts = ['jagatsinghpur', 'nabarangapur']

def osrm_route(src, dst):
    url = f"http://localhost:5000/route/v1/driving/{src.x},{src.y};{dst.x},{dst.y}?overview=false"
    try:
        r = requests.get(url)
        data = r.json()
        if r.status_code == 200 and 'routes' in data:
            return data['routes'][0]['duration'], data['routes'][0]['distance']
    except:
        return None, None
    return None, None

for district in districts:
    print(f"\n🚀 Processing: {district.upper()}")

    v_path = os.path.join(VILLAGE_DIR, f"{district}_villages_snapped.geojson")
    f_path = os.path.join(FACILITY_DIR, f"{district}_sub_cen.geojson")

    if not os.path.exists(v_path) or not os.path.exists(f_path):
        print(f"❌ Missing files for {district}, skipping...")
        continue

    villages = gpd.read_file(v_path).to_crs(epsg=4326)
    facilities = gpd.read_file(f_path).to_crs(epsg=4326)

    output_rows = []

    for i, v_row in tqdm(villages.iterrows(), total=len(villages), desc=f"⏱ Routing villages in {district}"):
        v_geom = v_row.geometry
        v_id = v_row.get('village_id', i)

        fac_proj = facilities.to_crs(epsg=32644)
        v_proj = gpd.GeoSeries([v_geom], crs=4326).to_crs(epsg=32644).iloc[0]
        fac_proj['euclid_dist'] = fac_proj.geometry.distance(v_proj)
        nearest = facilities.loc[fac_proj.sort_values('euclid_dist').index[:3]]

        best_time = float('inf')
        best_record = None

        for _, f_row in nearest.iterrows():
            f_geom = f_row.geometry
            duration, distance = osrm_route(v_geom, f_geom)
            if duration is not None and duration < best_time:
                best_time = duration
                best_record = {
                    'village_id': v_id,
                    'facility_id': f_row.get('facilityname', ''),
                    'facility_type': 'sub_cen',
                    'duration_sec': duration,
                    'distance_m': distance,
                    'facility_lat': f_geom.y,
                    'facility_lon': f_geom.x
                }

        if best_record:
            output_rows.append(best_record)

    df_out = pd.DataFrame(output_rows)
    out_path = os.path.join(OUTPUT_DIR, f"{district}_village_to_sub_cen.csv")
    df_out.to_csv(out_path, index=False)
    print(f"✅ Saved: {out_path} [{len(df_out)} records]")


🚀 Processing: JAGATSINGHPUR


⏱ Routing villages in jagatsinghpur: 100%|██████████| 1300/1300 [00:57<00:00, 22.63it/s]


✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\jagatsinghpur_village_to_sub_cen.csv [1300 records]

🚀 Processing: NABARANGAPUR


⏱ Routing villages in nabarangapur: 100%|██████████| 943/943 [00:37<00:00, 25.31it/s]

✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\nabarangapur_village_to_sub_cen.csv [943 records]


In [ ]:
import os
import geopandas as gpd
import pandas as pd
import requests
from tqdm import tqdm

# === CONFIG ===
FACILITY_TYPES = {
    'chc': 'chc',
    's_t_h': 's_t_h'
}

SNAPPED_VILLAGE_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_districts"
SNAPPED_FACILITY_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_Facilities"
OUTPUT_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts"
os.makedirs(OUTPUT_DIR, exist_ok=True)

DISTRICTS = [f.split('_')[0] for f in os.listdir(SNAPPED_VILLAGE_DIR) if f.endswith(".geojson")]
DISTRICTS = sorted(set(DISTRICTS))  # deduplicate

# === OSRM Routing Function ===
def osrm_route(src, dst):
    url = f"http://localhost:5000/route/v1/driving/{src.x},{src.y};{dst.x},{dst.y}?overview=false"
    try:
        r = requests.get(url)
        data = r.json()
        if r.status_code == 200 and 'routes' in data:
            return data['routes'][0]['duration'], data['routes'][0]['distance']
    except:
        return None, None
    return None, None

# === MAIN ROUTING ===
for facility_type, f_key in FACILITY_TYPES.items():
    print(f"\n🔁 Starting Routing: {facility_type.upper()}")

    for district in DISTRICTS:
        print(f"\n🚀 Processing {district.upper()} [{facility_type}]")

        # Paths
        v_path = os.path.join(SNAPPED_VILLAGE_DIR, f"{district}_villages_snapped.geojson")
        f_path = os.path.join(SNAPPED_FACILITY_DIR, f"{district}_{f_key}.geojson")
        out_path = os.path.join(OUTPUT_DIR, f"{district}_village_to_{f_key}.csv")

        if not os.path.exists(v_path) or not os.path.exists(f_path):
            print(f"❌ Missing files for {district} [{f_key}], skipping...")
            continue

        # Load
        villages = gpd.read_file(v_path).to_crs(epsg=4326)
        facilities = gpd.read_file(f_path).to_crs(epsg=4326)

        rows = []

        for i, v_row in tqdm(villages.iterrows(), total=len(villages), desc=f"⏱ Routing villages in {district}"):
            v_geom = v_row.geometry
            v_id = v_row.get('village_id', i)

            # Filter top-3 by Euclidean distance in projected CRS
            fac_proj = facilities.to_crs(epsg=32644)
            v_proj = gpd.GeoSeries([v_geom], crs=4326).to_crs(epsg=32644).iloc[0]
            fac_proj['euclid_dist'] = fac_proj.geometry.distance(v_proj)
            nearest = facilities.loc[fac_proj.sort_values('euclid_dist').index[:3]]

            # Route to top-3, keep best
            best_time = float('inf')
            best_record = None

            for _, f_row in nearest.iterrows():
                f_geom = f_row.geometry
                duration, distance = osrm_route(v_geom, f_geom)
                if duration is not None and duration < best_time:
                    best_time = duration
                    best_record = {
                        'village_id': v_id,
                        'facility_id': f_row.get('facilityname', ''),
                        'facility_type': f_key,
                        'duration_sec': duration,
                        'distance_m': distance,
                        'facility_lat': f_geom.y,
                        'facility_lon': f_geom.x
                    }

            if best_record:
                rows.append(best_record)

        df = pd.DataFrame(rows)
        df.to_csv(out_path, index=False)
        print(f"✅ Saved: {out_path} [{len(df)} records]")

In [33]:
import os
import pandas as pd

# 📁 Folder with routing outputs
OUTPUT_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts"

# 🏙️ All 30 standardized districts
districts = sorted([
    'anugul', 'balangir', 'baleshwar', 'bargarh', 'bhadrak', 'boudh', 'cuttack', 'deogarh',
    'dhenkanal', 'gajapati', 'ganjam', 'jagatsinghpur', 'jajapur', 'jharsuguda', 'kalahandi',
    'kandhamal', 'kendrapara', 'keonjhar', 'khordha', 'koraput', 'malkangiri', 'mayurbhanj',
    'nabarangapur', 'nayagarh', 'nuapada', 'puri', 'rayagada', 'sambalpur', 'sonepur', 'sundargarh'
])

# 🚑 Facility types you're routing for
facility_types = ['phc', 'chc', 'hwc', 'sdh', 'sth', 's_t_h']

In [ ]:
# 📝 Record existence status for each district + facility type
records = []

for district in districts:
    for ftype in facility_types:
        filename = f"{district}_village_to_{ftype}.csv"
        full_path = os.path.join(OUTPUT_DIR, filename)
        records.append({
            "district": district,
            "facility_type": ftype,
            "file_name": filename,
            "exists": os.path.exists(full_path)
        })

# 📊 Create DataFrame
df_routing_check = pd.DataFrame(records)
df_routing_check.head()

In [ ]:
# ❌ Filter missing files
missing = df_routing_check[df_routing_check['exists'] == False]

print(f"❌ Total missing routing files: {len(missing)}")
display(missing[['district', 'facility_type', 'file_name']])

# 📈 Summary by facility type
summary = df_routing_check.groupby('facility_type')['exists'].sum().reset_index()
summary.columns = ['facility_type', 'files_found']
summary['expected'] = len(districts)
summary['missing'] = summary['expected'] - summary['files_found']

print("\n📊 Routing coverage by facility type:")
display(summary)

In [36]:
import os
import pandas as pd

ROUTING_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts"

districts = sorted([
    'anugul', 'balangir', 'baleshwar', 'bargarh', 'bhadrak', 'boudh', 'cuttack', 'deogarh',
    'dhenkanal', 'gajapati', 'ganjam', 'jagatsinghpur', 'jajapur', 'jharsuguda', 'kalahandi',
    'kandhamal', 'kendrapara', 'keonjhar', 'khordha', 'koraput', 'malkangiri', 'mayurbhanj',
    'nabarangapur', 'nayagarh', 'nuapada', 'puri', 'rayagada', 'sambalpur', 'sonepur', 'sundargarh'
])

In [ ]:
# Check for sub_cen routing files
subcen_records = []

for district in districts:
    filename = f"{district}_village_to_sub_cen.csv"
    exists = os.path.exists(os.path.join(ROUTING_DIR, filename))
    subcen_records.append({
        "district": district,
        "file": filename,
        "exists": exists
    })

df_subcen_check = pd.DataFrame(subcen_records)

missing_subcen = df_subcen_check[df_subcen_check['exists'] == False]

print("✅ HWC (sub_cen) routing status:")
display(df_subcen_check)

print(f"\n❌ Missing sub_cen routing files: {len(missing_subcen)}")
display(missing_subcen[['district', 'file']])

In [ ]:
# Check for phc routing files
phc_records = []

for district in districts:
    filename = f"{district}_village_to_phc.csv"
    exists = os.path.exists(os.path.join(ROUTING_DIR, filename))
    phc_records.append({
        "district": district,
        "file": filename,
        "exists": exists
    })

df_phc_check = pd.DataFrame(phc_records)

missing_phc = df_phc_check[df_phc_check['exists'] == False]

print("✅ PHC routing status:")
display(df_phc_check)

print(f"\n❌ Missing PHC routing files: {len(missing_phc)}")
display(missing_phc[['district', 'file']])

In [40]:
import os
import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from tqdm import tqdm

# === PATH SETUP ===
VILLAGE_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_districts"
PHC_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Snapped_Facilities"
OUTPUT_DIR = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === DISTRICTS TO PROCESS ===
districts = ['boudh', 'jagatsinghpur', 'nabarangapur']

# === OSRM routing function ===
def osrm_route(src, dst):
    url = f"http://localhost:5000/route/v1/driving/{src.x},{src.y};{dst.x},{dst.y}?overview=false"
    try:
        r = requests.get(url)
        data = r.json()
        if r.status_code == 200 and 'routes' in data:
            duration = data['routes'][0]['duration']  # in seconds
            distance = data['routes'][0]['distance']  # in meters
            return duration, distance
    except:
        return None, None
    return None, None

# === PROCESS EACH DISTRICT ===
for district in districts:
    print(f"\n🚀 Processing PHC: {district.upper()}")

    # === Load village and PHC files ===
    v_path = os.path.join(VILLAGE_DIR, f"{district}_villages_snapped.geojson")
    f_path = os.path.join(PHC_DIR, f"{district}_phc.geojson")

    if not os.path.exists(v_path) or not os.path.exists(f_path):
        print(f"❌ Missing files for {district}, skipping...")
        continue

    villages = gpd.read_file(v_path).to_crs(epsg=4326)
    phcs = gpd.read_file(f_path).to_crs(epsg=4326)

    # === Prepare output rows ===
    output_rows = []

    for i, v_row in tqdm(villages.iterrows(), total=len(villages), desc=f"⏱ Routing villages in {district}"):
        v_geom = v_row.geometry
        v_id = v_row.get('village_id', i)

        # === Top 3 closest PHCs by Euclidean distance
        phcs['euclid_dist'] = phcs.geometry.distance(v_geom)
        nearest = phcs.sort_values('euclid_dist').head(3)

        best_time = float('inf')
        best_record = None

        for _, f_row in nearest.iterrows():
            f_geom = f_row.geometry
            duration, distance = osrm_route(v_geom, f_geom)
            if duration is not None and duration < best_time:
                best_time = duration
                best_record = {
                    'village_id': v_id,
                    'facility_id': f_row.get('facilityname', ''),
                    'facility_type': 'phc',
                    'duration_sec': duration,
                    'distance_m': distance,
                    'facility_lat': f_geom.y,
                    'facility_lon': f_geom.x
                }

        if best_record:
            output_rows.append(best_record)

    # === Save output ===
    df_out = pd.DataFrame(output_rows)
    out_path = os.path.join(OUTPUT_DIR, f"{district}_village_to_phc.csv")
    df_out.to_csv(out_path, index=False)
    print(f"✅ Saved: {out_path} [{len(df_out)} records]")


🚀 Processing PHC: BOUDH


⏱ Routing villages in boudh:   0%|          | 0/1233 [00:00<?, ?it/s]C:\Users\utkar\AppData\Local\Temp\ipykernel_78684\1940659376.py:54: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  phcs['euclid_dist'] = phcs.geometry.distance(v_geom)
⏱ Routing villages in boudh:   0%|          | 1/1233 [00:00<08:12,  2.50it/s]C:\Users\utkar\AppData\Local\Temp\ipykernel_78684\1940659376.py:54: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  phcs['euclid_dist'] = phcs.geometry.distance(v_geom)
⏱ Routing villages in boudh: 100%|██████████| 1233/1233 [01:01<00:00, 20.11it/s]


✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\boudh_village_to_phc.csv [1233 records]

🚀 Processing PHC: JAGATSINGHPUR


⏱ Routing villages in jagatsinghpur:   0%|          | 0/1300 [00:00<?, ?it/s]C:\Users\utkar\AppData\Local\Temp\ipykernel_78684\1940659376.py:54: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  phcs['euclid_dist'] = phcs.geometry.distance(v_geom)
C:\Users\utkar\AppData\Local\Temp\ipykernel_78684\1940659376.py:54: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  phcs['euclid_dist'] = phcs.geometry.distance(v_geom)
⏱ Routing villages in jagatsinghpur: 100%|██████████| 1300/1300 [01:12<00:00, 17.93it/s]


✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\jagatsinghpur_village_to_phc.csv [1300 records]

🚀 Processing PHC: NABARANGAPUR


⏱ Routing villages in nabarangapur:   0%|          | 0/943 [00:00<?, ?it/s]C:\Users\utkar\AppData\Local\Temp\ipykernel_78684\1940659376.py:54: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  phcs['euclid_dist'] = phcs.geometry.distance(v_geom)
C:\Users\utkar\AppData\Local\Temp\ipykernel_78684\1940659376.py:54: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  phcs['euclid_dist'] = phcs.geometry.distance(v_geom)
⏱ Routing villages in nabarangapur: 100%|██████████| 943/943 [00:49<00:00, 19.12it/s]

✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Policy note\Outputs\Districts\nabarangapur_village_to_phc.csv [943 records]
